In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import corner 
import matplotlib.lines as mlines
import matplotlib.ticker as mtick
import matplotlib.colors as color 
from chainconsumer import ChainConsumer
#from matplotlib import rcParams
from matplotlib import rc
import h5py
from lisaconstants import GM_SUN,c,ASTRONOMICAL_YEAR,ASTRONOMICAL_UNIT
import emcee
import matplotlib as mpl

# Read emcee chain sample file with h5py

In [ ]:
test_name='Keplerian Orbit Model'
test_name_for_file = 'Keplerian_Orbit_Model'
reader = emcee.backends.HDFBackend("samples_Keplerian_chain_omega_emcee_backend_testing.h5", read_only=True)
dir = './'

# Use emcee built-in functions to get autocorrelation time $\tau$.

In [ ]:

try:
    tau = reader.get_autocorr_time()
except:
    print('not accurate tau at all. Just for plotting purposes')
    tau =500


print('tau')
print(tau)


# Get Burn-in and thin values given autocorrelation time and load the chain with those settings. See emcee documention https://emcee.readthedocs.io/en/stable/

In [ ]:
burnin = int(2 * np.max(tau))
thin = int(0.5 * np.min(tau))
flatchain = reader.get_chain(discard=burnin,thin=thin,flat=True)
log_prob = reader.get_log_prob(discard=burnin,thin=thin,flat=True)

# Parameters of chain 

In [ ]:
removed = 0 #Burn-in already applied above.
minit1 = flatchain.T[0][removed::]
semi_major = flatchain.T[1][removed::]
arg_per = flatchain.T[2][removed::]

# Truth values of chain. Check your LISA Orbits simulation to make sure these are correct.

In [ ]:
arg_per_truth = -np.pi/2.0
minit1_truth = 0.0
semi_major_truth = ASTRONOMICAL_UNIT

# Array format for chain consumer

In [ ]:
data_plot = np.array([minit1,semi_major-semi_major_truth,arg_per-arg_per_truth]).T

# Use chain consumer to plot corner plots of parameter values. See documentation for arguments. https://samreay.github.io/ChainConsumer/

In [ ]:
c = ChainConsumer()

parameters=[r"$M_{1_{0}} (\mathrm{rad})$",r"$\Delta a (\mathrm{m})$",r"$\Delta\omega (\mathrm{rad})$"] #parameter labels


c.add_chain(data_plot, parameters=parameters,color=color.to_hex('blue'),name=test_name)
c.configure(summary_area=0.95,spacing=1.0,kde=False,smooth=1,shade=True,usetex=True,label_font_size=15,linewidths=1.5) #See chain conumer documention
c.configure_truth(color='k')
c.plotter.plot(chains=[test_name],figsize=(8.6,8.6),filename=dir+'corner_plot_Keplerian_Parameters_{0}.png'.format(test_name_for_file), truth=[minit1_truth,0.0,0.0],display=True,legend=True)
c.plotter.plot_walks(display=True)
plt.close()